In [11]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Flatten, Dropout, BatchNormalization
from keras.preprocessing import image
from keras.preprocessing import image_dataset_from_directory
from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input


In [12]:
#Loading data

train_data = image_dataset_from_directory(
    directory="../data/SkinDisease/train",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(224,224),
    batch_size=32

)

test_data = image_dataset_from_directory(
    directory="../data/SkinDisease/test",
    labels="inferred",
    label_mode='categorical',
    color_mode='rgb',
    image_size=(224,224),
    batch_size=32,
    shuffle=False
)

Found 13898 files belonging to 22 classes.
Found 1546 files belonging to 22 classes.


In [13]:
# Preprocessing images
def preprocess(image, label):
    image = preprocess_input(image)

    return image, label

train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)


In [14]:
train_data = train_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

In [15]:
augment_data = Sequential([
    
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomFlip('vertical'),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomTranslation(0.2, 0.2),
    keras.layers.RandomShear(x_factor=0.05, y_factor=0.05, fill_mode='constant')

])

In [16]:
base_model = EfficientNetB0(weights='imagenet',include_top=False, input_shape=(224,224,3))

In [17]:
base_model.trainable = False

# for layer in base_model.layers[:-20]:
#     layer.trainable = False

In [18]:
# Building model
model = Sequential()

model.add(keras.layers.Input(shape=(224,224,3)))
model.add(augment_data)
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(22, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), 
              metrics=["accuracy"])

In [20]:
history = model.fit(train_data, epochs=20, validation_data=test_data)

Epoch 1/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 624s 1s/step - accuracy: 0.1568 - loss: 3.4918 - val_accuracy: 0.2891 - val_loss: 2.6681
Epoch 2/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 518s 1s/step - accuracy: 0.2349 - loss: 2.9639 - val_accuracy: 0.3234 - val_loss: 2.5268
Epoch 3/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 519s 1s/step - accuracy: 0.2810 - loss: 2.7523 - val_accuracy: 0.3512 - val_loss: 2.4305
Epoch 4/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 552s 1s/step - accuracy: 0.2985 - loss: 2.6429 - val_accuracy: 0.3797 - val_loss: 2.3705
Epoch 5/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 524s 1s/step - accuracy: 0.3160 - loss: 2.5584 - val_accuracy: 0.3790 - val_loss: 2.3309
Epoch 6/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 516s 1s/step - accuracy: 0.3299 - loss: 2.5097 - val_accuracy: 0.3913 - val_loss: 2.2973
Epoch 7/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 512s 1s/step - accuracy: 0.3462 - loss: 2.4565 - val_accuracy: 0.4023 - val_loss: 2.2714
Epoch 8/20
435/435 ━━━━━━━━━━━━━━━━━━━━ 551s 1s/step - accuracy: 0.3604 - loss: 2.4020 - val_accu